In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize  import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("omw-1.4")
from ast import literal_eval 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
data= pd.read_csv(r"C:\Users\dell\Desktop\dataset\Hotel_Reviews.csv")

In [4]:
data.shape

(515738, 17)

In [5]:
# for simplicty will change the countroes name to Alpgha-2 codes
data.Hotel_Address = data.Hotel_Address.str.replace("Netherlands", "NL")
data.Hotel_Address = data.Hotel_Address.str.replace("United Kingdom", "UK")
data.Hotel_Address = data.Hotel_Address.str.replace("France", "FR")
data.Hotel_Address = data.Hotel_Address.str.replace("Italy", "IT")
data.Hotel_Address = data.Hotel_Address.str.replace("Spain", "SP")
data.Hotel_Address = data.Hotel_Address.str.replace("Austria", "AT")

In [6]:
#  will add or deriv a new cloumn from last string at ho0tel address wich specify where the hotel loctated 
# data["countries"] =[ x[-2:] for x in data.Hotel_Address]
data["countries"] = data.Hotel_Address.apply(lambda x: x.split(' ')[-1])

In [12]:
# to make sure that Counties added will use unique() to retrieve unique words at the column  
data.countries.unique()

array(['NL', 'UK', 'FR', 'SP', 'IT', 'AT'], dtype=object)

In [13]:
# will show all columns to drop needn't data 
data.columns

Index(['Hotel_Address', 'Average_Score', 'Hotel_Name', 'Tags', 'countries'], dtype='object')

In [15]:
# will work in new dataset
data.drop(['Additional_Number_of_Scoring', 'Review_Date',
       'Reviewer_Nationality',
       'Negative_Review', 'Review_Total_Negative_Word_Counts',
       'Total_Number_of_Reviews', 'Positive_Review',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score',
       'days_since_review', 'lat', 'lng'],axis=1,inplace=True)

KeyError: "['Additional_Number_of_Scoring', 'Review_Date', 'Reviewer_Nationality', 'Negative_Review', 'Review_Total_Negative_Word_Counts', 'Total_Number_of_Reviews', 'Positive_Review', 'Review_Total_Positive_Word_Counts', 'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score', 'days_since_review', 'lat', 'lng'] not found in axis"

In [10]:
data.columns

Index(['Hotel_Address', 'Average_Score', 'Hotel_Name', 'Tags', 'countries'], dtype='object')

In [11]:
# create a function help us to convert object(pandas.core.series.Series) to List then to Sring from Tags
def convert(column):
    column = column[0]
    if (type(column) != list):
        return "".join(literal_eval(column))
    else:
        return column    
dataset.head(6)

NameError: name 'dataset' is not defined

In [ ]:
# will call the convert function to change object dataType of column Tags to Srting 
# dataset['Tags'] = dataset[['Tags']].apply(convert, axis= 1)

In [ ]:
# To simplicty => change Countries and Tags columns to lowercase
data["Tags"]= data["Tags"].str.lower()
data["countries"]= data["countries"].str.lower()

In [ ]:
#  model
# location => Counties , description => Tags
def recomender(location , description):
#  lemmatiaztion for Tags column     
    description =  description.lower()
    word_tokenize(description)
    stop_words  =  stopwords.words("english")
    lemm  =  WordNetLemmatizer()
    
#     cheching there is a stopwords, if no will add at list 
    filtered= {word for word in description if not word in stop_words}
    filtered_set = set()
    for fs in filtered:
        filtered_set.add(lemm.lemmatize(fs))
        
        
#    creating varible refer to all countires == to location i want  
    country = data[data['countries']==location.lower()]
#     set index by using numpy                   shape[0]=> [0]> num of rows  
    country = country.set_index(np.arange(country.shape[0]))
    
    list1 = []
    list2 = [] 
    cos   = []
    
#     for loop to iteration of all rows    
    for i in range(country.shape[0]):
        
#     make tokinze and then, filter from stopwords
        temp_token = word_tokenize(country["Tags"][i])
        temp_set   = [word for word in temp_token if not word in stop_words]
#       list grouping the lemmatizetion of words tokenization  
        temp2_set =set()
        for xx in temp_set:
            temp2_set.add(lemm.lemmatize(xx))
        vector = temp2_set.intersection(filtered_set)
        cos.append(len(vector))
    country['similarity'] = cos
#     arange hotels by using tags rating 
    country = country.sort_values(by="similarity", ascending=False)
#                                   require            ,optional ^defult
    country.drop_duplicates(subset='Hotel_Name', keep="first", inplace= True)
    
    country.sort_values('Average_Score', ascending=False, inplace=True)
    country.reset_index(inplace = True)
    return country[["Hotel_Name", "Average_Score", "Hotel_Address"]].head(10)

In [ ]:
recomender('FR','Business trip')